In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")

from multi_group_simulation import MultiGroupSimulation
from stochastic_simulation import StochasticSimulation
from load_params import load_params

In [2]:
def single_group_config(pop_size, daily_contacts, test_rate, outside_infection_p):
    base_config_loc="/home/jmc678/covid_data/group-testing/src/simulations_v2/params/"\
                "june8params/noreopen/nominal_students.yaml"
    _, base_config = load_params(base_config_loc)
    base_config['test_population_fraction'] = test_rate
    base_config['expected_contacts_per_day'] = daily_contacts
    base_config['population_size'] = pop_size
    base_config['daily_outside_infection_p'] = outside_infection_p
    base_config['init_ID_prevalence_stochastic'] = True
    return base_config

def multi_group_config(pop_size, daily_contacts, test_rate, outside_infection_p, num_groups):
    base_configs = [single_group_config(pop_size / num_groups, 
                                        daily_contacts / num_groups, 
                                        test_rate, 
                                        outside_infection_p)] * num_groups
    interaction_matrix = np.matrix([[daily_contacts/num_groups] * num_groups] * num_groups)
    return base_configs, interaction_matrix

In [3]:
pop_size = 1000
daily_contacts = 10
test_rate = 0.2
outside_infection_p = 1.2e-5

num_groups = 10

single_grp_sim = StochasticSimulation(single_group_config(pop_size, daily_contacts, test_rate, outside_infection_p))

configs, interaction_mtx = multi_group_config(pop_size, daily_contacts, test_rate, outside_infection_p, num_groups)
multi_grp_sim = MultiGroupSimulation(configs, interaction_mtx)

In [4]:
sim_df = multi_grp_sim.run_new_trajectory(112)

In [5]:
def get_popsize(df):
    cols = df.columns
    subdf = df[[col for col in cols if 'cumulative' not in col]]
    return subdf.sum(axis=1).iloc[0]

def infection_pct(df):
    popsize = get_popsize(df)
    
    subdf = df[['cumulative_mild', 'cumulative_severe']]
    infections = subdf.sum(axis=1).iloc[-1]
    
    return infections / popsize * 100

def add_df_plot(df, label, color, alpha=0.1, linewidth=10):
    popsize = get_popsize(df)
    df[['S', 'QS']].apply(lambda x: (popsize - sum(x)) / popsize * 100, axis=1) \
                        .plot(label=label, alpha=alpha, linewidth=linewidth, color=color)

infection_pct(sim_df)

0.1

In [6]:
import cProfile
cProfile.run('multi_grp_sim.run_new_trajectory(10)')

         2605171 function calls (2477860 primitive calls) in 3.572 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2500    0.003    0.000    0.029    0.000 <__array_function__ internals>:2(all)
    12590    0.011    0.000    0.200    0.000 <__array_function__ internals>:2(any)
    12390    0.011    0.000    0.130    0.000 <__array_function__ internals>:2(atleast_1d)
    12390    0.014    0.000    0.539    0.000 <__array_function__ internals>:2(clip)
      220    0.000    0.000    0.004    0.000 <__array_function__ internals>:2(concatenate)
    24780    0.023    0.000    0.638    0.000 <__array_function__ internals>:2(extract)
    24780    0.019    0.000    0.150    0.000 <__array_function__ internals>:2(ndim)
    24780    0.018    0.000    0.090    0.000 <__array_function__ internals>:2(nonzero)
    24780    0.024    0.000    0.164    0.000 <__array_function__ internals>:2(place)
        9    0.000    0.000    0.000   

      210    0.000    0.000    0.006    0.000 missing.py:47(_isnull_new)
        9    0.000    0.000    0.000    0.000 missing.py:583(fill_zeros)
        9    0.000    0.000    0.000    0.000 missing.py:73(clean_fill_method)
        1    0.000    0.000    0.244    0.244 multi_group_simulation.py:40(reset_initial_state)
        1    0.000    0.000    3.572    3.572 multi_group_simulation.py:45(run_new_trajectory)
      900    0.013    0.000    0.032    0.000 multi_group_simulation.py:56(get_free_total)
     1800    0.004    0.000    0.029    0.000 multi_group_simulation.py:67(get_free_infectious)
       10    0.005    0.001    3.325    0.332 multi_group_simulation.py:90(step)
      220    0.000    0.000    0.000    0.000 multiarray.py:145(concatenate)
     1000    0.001    0.000    0.001    0.000 numeric.py:1782(isscalar)
    17498    0.011    0.000    0.021    0.000 numerictypes.py:293(issubclass_)
     8749    0.010    0.000    0.032    0.000 numerictypes.py:365(issubdtype)
      220 